<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/Catboost_20240825.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import copy
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

# warningsを非表示にする
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:

!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.4 MB/s eta 0:00:00


In [4]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [5]:

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

In [6]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_train5.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_test5.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [7]:
df_train.head()

,id,Age,TypeofContact,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,Passport,MonthlyIncome,ProdTaken,OwnCar,NumberOfChildren,HasChildren,CityTier_1,CityTier_2,CityTier_3,Occupation_0,Occupation_1,Occupation_2,ProductPitched_BASIC,ProductPitched_DELUXE,ProductPitched_KING,ProductPitched_STANDARD,ProductPitched_SUPERDELUXE,PreferredPropertyStar_3.0,PreferredPropertyStar_4.0,PreferredPropertyStar_5.0,PitchSatisfactionScore_1,PitchSatisfactionScore_2,PitchSatisfactionScore_3,PitchSatisfactionScore_4,PitchSatisfactionScore_5,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚,FamilySize,IncomePerAge,IncomePerFamilySize,IncomePerTrips,PersonTripsProduct,IncomePerChild,LogMonthlyIncome,PitchFollowupInteraction,TripsPerAge
0,0,50.00000,1,15.0,0,1.0,4.0,5.0,1,253,1.0,0,0.0,0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,5.060000,126.5,50.6,5.0,inf,5.537334,60.0,0.100000
1,1,56.00000,0,14.0,0,1.0,4.0,2.0,1,404,0.0,1,0.0,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,7.214286,404.0,202.0,2.0,inf,6.003887,56.0,0.035714
2,2,41.13181,1,10.0,1,1.0,3.0,4.0,0,278,1.0,0,0.0,0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,6.758759,139.0,69.5,4.0,inf,5.631212,30.0,0.097248
3,3,37.00000,1,18.0,1,1.0,3.0,1.0,0,326,0.0,1,0.0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,8.810811,326.0,326.0,1.0,inf,5.789960,54.0,0.027027
4,4,48.00000,0,17.0,1,1.0,3.0,4.0,0,258,1.0,1,0.0,0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,5.375000,258.0,64.5,4.0,inf,5.556828,51.0,0.083333


In [8]:
# Config cell
target = "ProdTaken"

In [21]:

train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [20]:
default_categorical_features = ['TypeofContact', 'CityTier', 'Occupation', 'Gender', 'ProductPitched', 'PreferredPropertyStar',
                                'Passport', 'PitchSatisfactionScore', 'Designation', 'MaritalStatus']
default_numerical_features = ['id','Age', 'DurationOfPitch', 'MonthlyIncome',
                            'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren','OwnCar'
                              ]
added_numerical_features = ['IncomePerAge','FamilySize','IncomePerFamilySize','IncomePerTrips','PersonTripsProduct','IncomePerChild','LogMonthlyIncome','PitchFollowupInteraction','TripsPerAge']
numerical_features = default_numerical_features + added_numerical_features



#特徴量の指定
features = df_train.columns.tolist()
#カテゴリカル特徴量の指定
categorical_features = copy.deepcopy(features)
print(f'numerical_features {numerical_features}')
for i in numerical_features:
    categorical_features.remove(i)
print(f'categorical_features {categorical_features}')

#学習に使用しない特徴量は以下で除外
RemoveList=[target, 'id']
for i in RemoveList:
    if i in numerical_features:
        numerical_features.remove(i)
        features.remove(i)
    elif i in categorical_features:
        categorical_features.remove(i)
        features.remove(i)
    else:
        features.remove(i)
print(f'features for training:{features}')

for col in categorical_features:
    df_train[col] = df_train[col].astype(int)
    df_test[col] = df_test[col].astype(int)

numerical_features ['id', 'Age', 'DurationOfPitch', 'MonthlyIncome', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren', 'OwnCar', 'IncomePerAge', 'FamilySize', 'IncomePerFamilySize', 'IncomePerTrips', 'PersonTripsProduct', 'IncomePerChild', 'LogMonthlyIncome', 'PitchFollowupInteraction', 'TripsPerAge']
categorical_features ['TypeofContact', 'Gender', 'Passport', 'ProdTaken', 'HasChildren', 'CityTier_1', 'CityTier_2', 'CityTier_3', 'Occupation_0', 'Occupation_1', 'Occupation_2', 'ProductPitched_BASIC', 'ProductPitched_DELUXE', 'ProductPitched_KING', 'ProductPitched_STANDARD', 'ProductPitched_SUPERDELUXE', 'PreferredPropertyStar_3.0', 'PreferredPropertyStar_4.0', 'PreferredPropertyStar_5.0', 'PitchSatisfactionScore_1', 'PitchSatisfactionScore_2', 'PitchSatisfactionScore_3', 'PitchSatisfactionScore_4', 'PitchSatisfactionScore_5', 'Designation_AVP', 'Designation_Executive', 'Designation_Manager', 'Designation_Senior Manager', 'Designation_VP', 'MaritalStatu

In [11]:
cols_category=categorical_features

In [12]:

params_cat = {
    "iterations": 1000,  # n_estimatorsに相当
    'eval_metric' : 'AUC',
    'depth':1,
    'l2_leaf_reg' : 8,
    "learning_rate": 0.1,
    "colsample_bylevel": 0.8,  # colsample_bytreeに相当
    "random_seed": 0,
    "verbose": 100,  # 学習の進捗を100回毎に表示
    "use_best_model": True,  # 早期停止のための設定
}

In [22]:

# f1スコアが最も高くなる点を見つける
from sklearn import metrics
def decide_cutoff(val_y, preds_y_proba):
    mean_f1_list = []
    fpr, tpr, thresholds = metrics.roc_curve(val_y, preds_y_proba)
    for threshold in thresholds:
        preds_y = [1 if prob > threshold else 0 for prob in preds_y_proba]
        #preds_y=preds_y_proba
        mean_f1_list.append(f1_score(val_y, preds_y, average='macro'))
    return np.max(mean_f1_list), thresholds[np.argmax(mean_f1_list)]

In [43]:
list_metrics_auc = []
list_metrics_f1 = []
list_cutoff = []
list_models = []
fraction_of_positives_list = []
mean_predicted_value_list = []

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for fold, (trn_idx, val_idx) in enumerate(cv.split(train_x, train_y), start=1):
    trn_x = train_x.iloc[trn_idx, :]
    trn_y = train_y[trn_idx]
    val_x = train_x.iloc[val_idx, :]
    val_y = train_y[val_idx]
    train_pool = Pool(data=trn_x, label=trn_y, cat_features=cols_category)
    val_pool = Pool(data=val_x, label=val_y, cat_features=cols_category)

    model_cat = CatBoostClassifier(**params_cat, early_stopping_rounds=100)
    model_cat.fit(train_pool, eval_set=val_pool)

    list_models.append(model_cat)
    preds_y_proba = model_cat.predict_proba(val_pool)[:, 1]
    auc = roc_auc_score(val_y, preds_y_proba)
    f1, threshold = decide_cutoff(val_y, preds_y_proba)
    list_metrics_auc.append(auc)
    list_metrics_f1.append(f1)
    list_cutoff.append(threshold)
    print(f"Fold: {fold}, AUC: {auc}, f1 score: {f1} Threshold: {threshold}")

        # キャリブレーション曲線を計算
    fraction_of_positives, mean_predicted_value = calibration_curve(val_y, preds_y_proba, n_bins=10)
    fraction_of_positives_list.append(fraction_of_positives)
    mean_predicted_value_list.append(mean_predicted_value)

  # AUC
print(np.mean(list_metrics_auc), np.mean(list_metrics_f1), np.median(list_cutoff))

0:	test: 0.6914716	best: 0.6914716 (0)	total: 1.8ms	remaining: 1.8s
100:	test: 0.8814047	best: 0.8848161 (63)	total: 316ms	remaining: 2.82s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8848160535
bestIteration = 63

Shrink model to first 64 iterations.
Fold: 1, AUC: 0.8848160535117057, f1 score: 0.757638888888889 Threshold: 0.2912275507843148
0:	test: 0.6814381	best: 0.6814381 (0)	total: 1.39ms	remaining: 1.39s
100:	test: 0.7880268	best: 0.7890301 (79)	total: 236ms	remaining: 2.1s
200:	test: 0.7935117	best: 0.7939799 (190)	total: 426ms	remaining: 1.69s
300:	test: 0.7969900	best: 0.7977926 (290)	total: 665ms	remaining: 1.54s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7977926421
bestIteration = 290

Shrink model to first 291 iterations.
Fold: 2, AUC: 0.7977926421404682, f1 score: 0.6832809773123909 Threshold: 0.17267013422762675
0:	test: 0.6647492	best: 0.6647492 (0)	total: 1.67ms	remaining: 1.67s
100:	test: 0.8715719	best: 0.8715719 (100

In [ ]:
0.846302095420107

In [41]:
import pickle

In [42]:
# モデルを保存する
# 全てのモデルを保存する
for fold, model_cat in enumerate(list_models, start=1):
    with open(f'/content/drive/MyDrive/Signate/2024summer/models/catboost_fold_v3{fold}.pkl', 'wb') as f:
        pickle.dump(model_cat, f)

In [37]:


threshold = np.median(list_cutoff)
preds_y_proba = np.zeros(len(df_test))
for model in list_models:
    feature_names = model.feature_names_
    preds_y_proba += model.predict_proba(df_test[feature_names])[:, 1] / len(list_models)
    print(f"Model: {model}, Proba: {preds_y_proba}")  # 予測確率を表示
preds_y = preds_y_proba



Model: <catboost.core.CatBoostClassifier object at 0x799985087a90>, Proba: [0.00910519 0.01620988 0.02912276 ... 0.05231587 0.02915254 0.00310952]
Model: <catboost.core.CatBoostClassifier object at 0x79997696a200>, Proba: [0.01639023 0.03704359 0.06388634 ... 0.11220536 0.0553172  0.00388971]
Model: <catboost.core.CatBoostClassifier object at 0x799976957520>, Proba: [0.02232423 0.05536425 0.09120235 ... 0.16561271 0.0742028  0.00459033]
Model: <catboost.core.CatBoostClassifier object at 0x79998501d270>, Proba: [0.03010118 0.07435034 0.12611256 ... 0.22702704 0.10780413 0.00677869]
Model: <catboost.core.CatBoostClassifier object at 0x7999d11f6a10>, Proba: [0.03770555 0.0940031  0.15668034 ... 0.28508619 0.13550649 0.00816456]
Model: <catboost.core.CatBoostClassifier object at 0x7999d11f4d90>, Proba: [0.04473351 0.11377782 0.18397394 ... 0.34469685 0.15912441 0.00899764]
Model: <catboost.core.CatBoostClassifier object at 0x7999771e9ff0>, Proba: [0.0517075  0.13363177 0.21672615 ... 0.404

In [38]:
count = 10

In [39]:

from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/catboost_{timestamp}_{count:03d}.csv"
ss[1] = preds_y
ss.to_csv(file_name, header=False, index=False)